In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define dataset path
dataset_path = r"D:\intel-small"
# Data Preprocessing
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(250, 250),
    batch_size=32,
    class_mode="categorical",  # Ensures multi-class training
    subset="training"
)


# Load Validation Data
val_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(250, 250),
    batch_size=32,
    class_mode="categorical",
    subset="validation"
)

# Define CNN Model
model = Sequential([
    Conv2D(10, (3,3), activation='relu', input_shape=(250, 250, 3)),
    Conv2D(10, (3,3), activation='relu'),
    
    Conv2D(35, (3,3), activation='relu'),
    Conv2D(35, (3,3), activation='relu'),
    MaxPooling2D((5,5)),  # 10x10x35
    
    Conv2D(100, (3,3), activation='relu'),
    MaxPooling2D((2,2)),  # 2x2x100
    
    Flatten(),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(train_generator.num_classes, activation='softmax')  # Use "softmax" for multi-class
])

# Compile Model
model.compile(optimizer='adam',
              loss='binary_crossentropy',  # Change to categorical_crossentropy if multi-class
              metrics=['accuracy'])

# Train Model
model.fit(train_generator, validation_data=val_generator, epochs=10)

# Save Model
# model.save("cnn_model.h5")

model.evaluate(train_generator)


Found 48 images belonging to 6 classes.
Found 12 images belonging to 6 classes.
Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - accuracy: 0.2083 - loss: 0.6201 - val_accuracy: 0.1667 - val_loss: 0.6046
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - accuracy: 0.1389 - loss: 0.5271 - val_accuracy: 0.1667 - val_loss: 0.4657
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - accuracy: 0.1944 - loss: 0.4657 - val_accuracy: 0.2500 - val_loss: 0.4742
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - accuracy: 0.4236 - loss: 0.4405 - val_accuracy: 0.3333 - val_loss: 0.4647
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 904ms/step - accuracy: 0.3611 - loss: 0.4094 - val_accuracy: 0.2500 - val_loss: 0.4581
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - accuracy: 0.5347 - loss: 0.3608 - val_accuracy: 0.1667 - val_loss: 0.4419
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - accuracy: 0.8056 - loss: 0.2876 - val_accuracy: 0.0833 - val_loss: 0.4819
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accurac

[0.10159042477607727, 0.9791666865348816]

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Path to the new image
new_image_path = r"D:\Intel-Image-Data\seg_test\seg_test\street\20126.jpg"
new_image_path = r"D:\Intel-Image-Data\seg_train\seg_train\forest\175.jpg"

# Load and preprocess the image
img = image.load_img(new_image_path, target_size=(250, 250))  # Resize to match model input
img_array = image.img_to_array(img) / 255.0  # Normalize
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension


# Predict class probabilities
predictions = model.predict(img_array)

# Get the class with the highest probability
predicted_class = np.argmax(predictions, axis=1)[0]

# Load class labels
class_labels = {v: k for k, v in train_generator.class_indices.items()}  # Reverse mapping

# Print the predicted class
print("Predicted class:", class_labels[predicted_class])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
Predicted class: street


### resnet

In [ ]:
from tensorflow.keras.applications import ResNet50, EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D

num_classes = train_generator.num_classes

# Load ResNet50 (pretrained on ImageNet)
base_model_resnet = ResNet50(weights="imagenet", include_top=False, input_shape=(250, 250, 3))

# Freeze base model layers
base_model_resnet.trainable = False

# Add custom classification head
x = GlobalAveragePooling2D()(base_model_resnet.output)
x = Dense(256, activation="relu")(x)
x = Dense(128, activation="relu")(x)
output = Dense(num_classes, activation="softmax")(x)

# Create model
resnet_model = Model(inputs=base_model_resnet.input, outputs=output)

# Compile model
resnet_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Train model
resnet_model.fit(train_generator, validation_data=val_generator, epochs=10)

# Save model
# resnet_model.save("resnet_model.h5")


### efficient net

In [ ]:
# Load EfficientNetB0 (pretrained on ImageNet)
base_model_efficientnet = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(250, 250, 3))

# Freeze base model layers
base_model_efficientnet.trainable = False

# Add custom classification head
x = GlobalAveragePooling2D()(base_model_efficientnet.output)
x = Dense(256, activation="relu")(x)
x = Dense(128, activation="relu")(x)
output = Dense(num_classes, activation="softmax")(x)

# Create model
efficientnet_model = Model(inputs=base_model_efficientnet.input, outputs=output)

# Compile model
efficientnet_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Train model
efficientnet_model.fit(train_generator, validation_data=val_generator, epochs=10)

# Save model
# efficientnet_model.save("efficientnet_model.h5")


# yolo

In [14]:
from ultralytics import YOLO

# Load YOLOv8 (pretrained on COCO dataset)
model = YOLO("yolov8s.pt")  # 's' = small, 'm' = medium, 'l' = large


ModuleNotFoundError: No module named 'ultralytics'

In [ ]:
# Perform object detection
results = model("path_to_your_image.jpg", save=True)

# Show detection results
results[0].show()


In [ ]:
# Run YOLO on a video
results = model("path_to_video.mp4", save=True)


In [ ]:
# Run YOLO on webcam
model.predict(source=0, show=True)
